In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 2.1MB 22.3MB/s 
     |████████████████████████████████| 1.2MB 35.1MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
     |████████████████████████████████| 901kB 62.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=da3fdb1ed4e0b89b4389ac440d9fe1a487f3f17882af911873c12e95c3211b5b
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [2]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-04 21:25:37--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-04 21:25:37--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd61e904455835cc44888c4f138.dl.dropboxusercontent.com/cd/0/inline/BN3elKVOJOA-pwsqB_lneVBabHSJhYV6c6GSmvZWlpxGLpC6cYQ0qL-wnRnuXlly98DSLywORn4ZeB2Og2tpozuW3mSHEmHC1QxRksEdJKmsP_M7tibQPV2GYdRjXRlwuHUNAAamVG6kLTCALJQuz2p6/file# [following]
--2021-05-04 21:25:38--  https://ucd61e904455835cc44888c4f138.dl.dropboxusercontent.com/cd/0/inline/BN3elKVOJOA-pwsqB_lneVBabHSJhYV6c6GSmvZWlpxGLpC6cYQ0qL-wnRnuX

In [3]:
!nvidia-smi

Tue May  4 21:26:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import pickle
from sentence_transformers import SentenceTransformer

import torch
!pip install transformers
from transformers import BertForNextSentencePrediction, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import sklearn
import collections
import random

from google.colab import drive

random.seed(2021)
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [90]:
# load sample data
# format { post_id: [post, [comment1, comment2, ... ] }

data_file_path = 'grouped_data.pickle'

def get_best_comm(comms):
  best_comm = None
  best_score = -np.inf

  for comm in comms:
    if comm['score'] > best_score:
      best_comm = comm
      best_score = comm['score']

    return best_comm

def load_data(file_path):
    posts_comments = []
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        for post_id, val in tqdm(list(data.items())):
            post = val[0]
            comms = val[1]
            # best_comm = get_best_comm(comms)

            # if len(post['title']) < 500 and len(best_comm['body']) < 500 and best_comm['body'] != '[deleted]' and best_comm['body'] != '[removed]':
            #   posts_comments.append((post['title'], best_comm['body']))

            for com in comms:
                try:
                    if len(post['title']) < 500 and len(com['body']) < 500 and com['body'] != '[deleted]' and com['body'] != '[removed]':
                        posts_comments.append((post['title'], com['body']))
                except:
                    pass
                
    return posts_comments
    
posts_comms = load_data(data_file_path)
len(posts_comms)

1403354

In [91]:
post_comms_realfake = []
labels = []
for post, comment in posts_comms:
  while True:
    rpost, rcomment = random.choice(posts_comms)
    if post != rpost:
      break
  
  post_comms_realfake.append((post, comment))
  labels.append(0) # comment \in post
  post_comms_realfake.append((post, rcomment))
  labels.append(1) # comment \nin post

len(post_comms_realfake)

2806708

In [92]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.25, shuffle=True)
#train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.01, train_size=.04, shuffle=True)

In [93]:
model = SentenceTransformer('stsb-distilroberta-base-v2')

In [94]:
train_posts = [item[0] + " " + item[1] for item in train_texts]
test_posts = [item[0] + " " + item[1] for item in test_texts]

In [95]:
embed_train = model.encode(train_posts)

In [96]:
embed_test = model.encode(test_posts)

In [97]:
# create dataset class and load encodings and associated labels to it

class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return torch.tensor(self.encodings[idx]), torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.labels)

train_dataset = upvote_prediction_dataset(embed_train, train_labels)
test_dataset = upvote_prediction_dataset(embed_test, test_labels)

In [98]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [99]:
class Net(torch.nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = torch.nn.Linear(input_size, 250)  
        self.relu = torch.nn.ReLU()                        
        self.fc2 = torch.nn.Linear(250, 50) 
        self.relu2 = torch.nn.ReLU()    
        self.fc3 = torch.nn.Linear(50, 2) 
        self.softmax = torch.nn.Softmax(dim=1) 
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out

In [100]:
net = Net(embed_train.shape[1])
net.cuda()

Net(
  (fc1): Linear(in_features=768, out_features=250, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=250, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [101]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

In [107]:
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for i, (x, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        if torch.cuda.is_available():
          x = x.cuda()
          labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(x)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss))

Epoch [1/10], Step [100/32891], Loss: 0.4977
Epoch [1/10], Step [200/32891], Loss: 0.4983
Epoch [1/10], Step [300/32891], Loss: 0.5454
Epoch [1/10], Step [400/32891], Loss: 0.4320
Epoch [1/10], Step [500/32891], Loss: 0.5278
Epoch [1/10], Step [600/32891], Loss: 0.4960
Epoch [1/10], Step [700/32891], Loss: 0.4831
Epoch [1/10], Step [800/32891], Loss: 0.4933
Epoch [1/10], Step [900/32891], Loss: 0.5123
Epoch [1/10], Step [1000/32891], Loss: 0.5601
Epoch [1/10], Step [1100/32891], Loss: 0.5040
Epoch [1/10], Step [1200/32891], Loss: 0.4343
Epoch [1/10], Step [1300/32891], Loss: 0.5581
Epoch [1/10], Step [1400/32891], Loss: 0.5041
Epoch [1/10], Step [1500/32891], Loss: 0.5909
Epoch [1/10], Step [1600/32891], Loss: 0.4496
Epoch [1/10], Step [1700/32891], Loss: 0.5450
Epoch [1/10], Step [1800/32891], Loss: 0.5686
Epoch [1/10], Step [1900/32891], Loss: 0.5652
Epoch [1/10], Step [2000/32891], Loss: 0.5171
Epoch [1/10], Step [2100/32891], Loss: 0.5005
Epoch [1/10], Step [2200/32891], Loss: 0.54

In [108]:
net.eval()
out = net.forward(torch.as_tensor(embed_test).cuda())

In [109]:
y_pred = out.cpu().detach().argmax(dim=1)
y_true = test_labels

In [110]:
print("Accuracy:",sklearn.metrics.accuracy_score(test_labels, y_pred))
print("Precision:",sklearn.metrics.precision_score(test_labels, y_pred))
print("Recall:",sklearn.metrics.recall_score(test_labels, y_pred))
print("f1-score:",sklearn.metrics.f1_score(test_labels, y_pred))

Accuracy: 0.7578686489652646
Precision: 0.7554056880557612
Recall: 0.7622440248702299
f1-score: 0.7588094501245006


In [111]:
torch.save(net.state_dict(), '/content/gdrive/My Drive/relevancy_model_v3')